In [2]:
import tensorflow as tf
from tensorflow.keras import datasets, models, layers
from sklearn.model_selection import train_test_split
import wandb
from wandb.keras import WandbCallback
from matplotlib import image
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
tf.config.list_physical_devices('GPU')

In [3]:
# A function to return a CNN model based on input hyperparameters
def createCNN(n_filters, filter_size, n_dense, num_conv_layers, input_shape, activation, n_output, filter_size_decay = 1, dropout_conv = 0.0, dropout_dense = 0.0, batch_normalize = False, data_augment = False):
    model = models.Sequential()
    
    if data_augment:
        data_augmentation = tf.keras.Sequential([ 
                            layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
                            layers.experimental.preprocessing.RandomRotation(0.2)])
        model.add(data_augmentation)
    
    if batch_normalize:
        model.add(layers.BatchNormalization(input_size=input_size))
    for ii in range(0, num_conv_layers): # Number of convolutional layers is num_conv_layers
        if ii == 0:
            model.add(layers.Conv2D(n_filters, (filter_size, filter_size), activation=activation, input_shape=input_shape))
        else:
            model.add(layers.Conv2D(n_filters, (filter_size, filter_size), activation=activation))
        model.add(layers.MaxPooling2D((2, 2)))
        if dropout_conv != 0.0:
            model.add(layers.Dropout(dropout_conv))
    
        n_filters = int(n_filters * filter_size_decay)
    
    model.add(layers.Flatten())
    model.add(layers.Dense(n_dense, activation=activation))
    if dropout_dense != 0.0:
        model.add(layers.Dropout(dropout_dense))
    model.add(layers.Dense(n_output))
    
    return model

In [ ]:
dataset_path = "../../nature_12K/inaturalist_12K"
train_path = dataset_path + "/train"
test_path = dataset_path + "/val"

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

max_shape = (32, 32, 3)

data_train = ImageDataGenerator(validation_split = 0.1)

data_test = ImageDataGenerator()

train_generator = data_train.flow(
        train_images,
        train_labels,
        subset = 'training')

validation_generator = data_train.flow(
        train_images,
        train_labels,
        subset = 'validation')

test_generator = data_train.flow(
        test_images,
        test_labels)

In [ ]:
my_model = my_cnn(16, 3, 64, (32, 32, 3), 'relu', 10)
my_model.summary()

In [4]:
dataset_path = "../../nature_12K/inaturalist_12K"
train_path = dataset_path + "/train"
test_path = dataset_path + "/val"
max_shape = (256, 256, 3)

data_train = ImageDataGenerator(validation_split = 0.1)#, rescale = 1. / 255)

data_test = ImageDataGenerator()#rescale = 1. / 255)

train_generator = data_train.flow_from_directory(
        train_path,
        class_mode='categorical',
        subset = 'training')

validation_generator = data_train.flow_from_directory(
        train_path,
        class_mode='categorical',
        subset = 'validation')

test_generator = data_test.flow_from_directory(
        test_path,
        class_mode='categorical')

Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


In [5]:
def train(model,
          optimizer,
          loss_fn,
          epochs=10):
    model.compile(optimizer=optimizer, loss=loss_fn, metrics = ['accuracy'])
    model.fit(train_generator, epochs=epochs, validation_data=validation_generator)#, callbacks=[WandbCallback()])

In [ ]:
sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'accuracy',
        'goal': 'maximize'
    }, 'parameters' : {
              "n_filters": {'values': [32, 64]},
              "filter_size_decay" : {'values':[0.5, 1, 1.15]},
              "data_augment": {'values':[True, False]},
              "dropout" : {'values':[0.0, 0.2, 0.4]},
              "batch_normalize" : {'values':[True, False]},
              "filter_size" : {'values':[3, 5]},
              "n_dense" : {'values':[50, 100]}
        }
    }

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity = '0x2e4', project = 'cs6910-a2')

In [ ]:
def run():
    default_config = {
              "n_filters": 32,
              "filter_size_decay" : 1,
              "data_augment": False,
              "dropout" : 0.0,
              "batch_normalize" : False,
              "filter_size" : 3,
              "n_dense" : 50
           }

    run = wandb.init(project='cs6910-a2', config=default_config)
    config = wandb.config

    # initialize model
    model = createCNN(n_filters = config.n_filters, filter_size = config.filter_size, n_dense = config.n_dense, num_conv_layers = 5, input_shape = max_shape, activation = 'relu', n_output = 10, filter_size_decay=config.filter_size_decay, dropout_conv = config.dropout / 3, dropout_dense = config.dropout, batch_normalize=config.batch_normalize, data_augment=config.data_augment)

    # Instantiate an optimizer to train the model.
    optimizer = tf.keras.optimizers.Nadam()
    # Instantiate a loss function.
    loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

    train(model,
      optimizer,
      loss_fn)

In [ ]:
wandb.agent(sweep_id, run)

In [ ]:
import random

a = [[[0 for k in range(4)] for i in range(4)] for j in range(15)]
b = [0 for k in range(15)]

for i in range(0, 15):
    for j in range(0, 4):
        for k in range(0, 4):
            a[i][j][k] = random.randint(0, 15)
    b[i] = random.randint(1, 3)

f = open('train.txt', 'w')
f.write(str(a))
f.close()
f = open('test.txt', 'w')
f.write(str(b))
f.close()

In [ ]:
import ast

f = open('train.txt', 'r')
train_s = np.asarray(ast.literal_eval(f.read()))
print(train_s)
f.close()

f = open('test.txt', 'r')
test_s = np.asarray(ast.literal_eval(f.read()))
print(test_s)
f.close()

In [ ]:
testing_model = createCNN(10, 2, 2, 3, (32, 32, 3), 'relu', 10)
testing_model.summary()

In [ ]:
testing_model.compile(optimizer='nadam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = testing_model.fit(X_train, y_train, epochs=10, 
                    validation_data=(X_val, y_val))

In [6]:
model = createCNN(n_filters = 32, filter_size = 3, n_dense = 100, num_conv_layers = 5, input_shape = max_shape, activation = 'relu', n_output = 10)

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 32)        9

In [ ]:
# Instantiate an optimizer to train the model.
optimizer = tf.keras.optimizers.Nadam()
# Instantiate a loss function.
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

train(model,
  optimizer,
  loss_fn)

Epoch 1/10
 46/282 [===>..........................] - ETA: 4:12 - loss: 3.7756 - accuracy: 0.1333